Finding Shortest Path Algorithm
=========

## 1. Reparing initial input

Get input vector of graph. A library with each element according vector and its length.

In [1]:
vectors = {'BA':1,'AC':2,'CB':2,'BD':3,
          'DC':1,'CE':1,'DE':1,'DF':2,
          'EF':2}

Creating $\phi$-matrix for every vectors

In [2]:
list_vec = list(vectors.keys())
# print(list_vec)
weights = [vectors[i] for i in list_vec]
# print(weights)

def FindPoints(list_vec):
    points = []
    tem_point = [points.extend(list(vector)) for vector in list_vec]
    points = list(set(points))
    #print(points)
    return points

points = FindPoints(list_vec)
print(points)

['E', 'B', 'A', 'F', 'D', 'C']


In [3]:
import numpy as np

def Cre_Matrix(vectors, points):
    matrixes = []
    for vector in vectors:
        pos_row = [0]*len(points)
        neg_row = [0]*len(points)

        pos_index = points.index(vector[1])
        neg_index = points.index(vector[0])

        pos_row[pos_index] += 1
        neg_row[neg_index] += 1

        matrixes.append(np.array([pos_row,neg_row]))
    return matrixes

matrixes = Cre_Matrix(list_vec, points)
print(matrixes)
        

[array([[0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]]), array([[0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0]]), array([[0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1]]), array([[0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0]]), array([[0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0]]), array([[1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1]]), array([[1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0]]), array([[0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0]]), array([[0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0]])]


## 2. Solve the shortest path

Using the [Solve Inequaltion](https://github.com/Truongphi20/inequaltion) to solve the problem. The conditions for establishing the system of inequalities are used as follows:

- Matrixes coefficient equal to 0 or 1
- Total path length is minima
- Sum of fixed coefficient equal to number of points minus 1
- Coefficients must match the final $\phi$-matrix

In [4]:
import sinequal as si

biens = ['a','b','c','d','e','f','g','h','i','M']
cuctri = [0,0,0,0,0,0,0,0,0,0]

he_bpt = ['a>=0','b>=0','c>=0','d>=0','e>=0','f>=0','g>=0','h>=0','i>=0',
         'a<=1','b<=1','c<=1','d<=1','e<=1','f<=1','g<=1','h<=1','i<=1', #coefficient equal 0 or 1
         '1*a+2*b+2*c+3*d+1*e+1*f+1*g+2*h+2*i-M==0', #total path length is minima
         'a+b+c+d+e+f+g+h+i==5', #Sum equal point number minus 1
          'a<=1','c<=1','b+e<=1','d<=1','f+g<=1','h+i<=1', #line 1 of matrix
          'b<=1','a+d<=1','c+f<=1','e+g+h<=1','i<=1', #line 2 of matrix
         ]

items_lay = si.Solve_Inequal(biens, cuctri, he_bpt)
print(items_lay)

(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'M'], [[1, 1, 1, 0, 0, 0, 1, 0, 1, 8], [0, 1, 1, 1, 0, 0, 1, 0, 1, 10]])


Get shortest length and coeffient

In [5]:
shortest_length = items_lay[1][0][-1]
print(shortest_length)

coeffs = [lis[:-1] for lis in items_lay[1]]
print(coeffs)

8
[[1, 1, 1, 0, 0, 0, 1, 0, 1], [0, 1, 1, 1, 0, 0, 1, 0, 1]]


In [6]:
def PhiMatrix(coeffs,points,matrixes):
    rs_matrix = []
    for coeff in coeffs:
        matrix_i = np.array([[0]*len(points),[0]*len(points)])
        for matrix, co in zip(matrixes,coeff):
            matrix_i += matrix*co
        rs_matrix.append(matrix_i)
    return rs_matrix

rs_matrix = PhiMatrix(coeffs,points,matrixes)
print(rs_matrix[0])
        

[[1 1 1 1 0 1]
 [1 1 1 0 1 1]]


We see, to remove noise results we need to establish following condition:
$$m+d-c=1$$

With:
- $m$ is number of enclosed plane
    + $m=\sum {{a_i}.{b_{_i}}}$
        - $a_i$ and $b_i$ are coefficients that have a prefix and suffix of are the same character. Ex: BA and CB
- $d$ is number of point
- $c$ is number of edge

In [7]:
def ChangeVec(took_vectors,start_char,path):
    for vector in took_vectors:
        if vector[0] == start_char:
            path += vector[-1]
            took_vectors.remove(vector)
    # print(path)
    # print(took_vectors)
    return path, took_vectors

def FindPath(coe, ma):

    took_vectors = [list_vec[i] for i in range(len(list_vec)) if coe[i] == 1]
    # print(took_vectors)

    start_index = list(ma[0]).index(0)
    start_char = points[start_index]
    # print(start_char)

    end_index = list(ma[1]).index(0)
    end_char = points[end_index]
    # print(end_char)

    path = start_char
    path, took_vectors = ChangeVec(took_vectors,start_char,path)
    # print(path)
    # print(took_vectors)

    while path[-1] != end_char:
        path, took_vectors = ChangeVec(took_vectors,path[-1],path)
    # print(path)
    return path


def ColectPath(coeffs,rs_matrix,points):
    final_paths =  []
    lene = len(points)
    for coe, ma in zip(coeffs,rs_matrix):
        path = FindPath(coe, ma)
        if len(path) == lene:
            final_paths.append(path)
    return final_paths


In [8]:
path = ColectPath(coeffs,rs_matrix,points)
print(path)

['ACBDEF']
